# link your Testdata:

In [1]:
link = '../data/test.csv'

In [2]:
import requests
import pandas as pd
import numpy as np
import sys
sys.path.append('..')
from util.helper import Filler, NewFeatures, Encoder, Dropper
df = pd.read_csv(link)

## ↓ Run this code for the whole cleaning-process: ↓

In [3]:
import pandas as pd
import numpy as np
import sys
sys.path.append('..')
from util.helper import Filler, NewFeatures, Encoder, Dropper
df = pd.read_csv(link)
ga_hta = pd.read_excel('../data/ga_hta.xlsx')
ga_hta = ga_hta[ga_hta['Jahr_An_Anno'] == ga_hta['Jahr_An_Anno'].max()]
ga_hta = ga_hta[['PLZ_NPA', 'GA_AG', 'HTA_ADT_meta-prezzo']]
ga_hta.columns = ['PLZ', 'Amount_GA', 'Amount_HT']
ga = ga_hta.set_index('PLZ').drop('Amount_HT', axis=1).to_dict()
ga = ga['Amount_GA']
df['Amount_GA'] = df['Zip'].replace(ga)
ht = ga_hta.set_index('PLZ').drop('Amount_GA', axis=1).to_dict()
ht = ht['Amount_HT']
df['Amount_HT'] = df['Zip'].replace(ht)
bahnhof = pd.read_html('https://de.wikipedia.org/wiki/Liste_der_Bahnh%C3%B6fe_in_der_Schweiz')
bahnhof  = pd.concat(bahnhof, axis=0)
bahnhof['has_bhf'] = 1
bahnhof = bahnhof[['Standortgemeinde', 'has_bhf']]
bahnhof.columns = ['Locality', 'has_bhf']
df['has_bhf'] = np.where(df['Locality'].isin(bahnhof['Locality'].unique()), 1, 0)
dropper = Dropper()
df = dropper.drop_all(df)
fill = Filler()
df = fill.fill_all(df=df, fill_method='0')
new_features = NewFeatures()
df = new_features.add(df)
df = new_features.combine_cat_features(df)
encoder = Encoder()
df = encoder.count_encoding(df)
print('Done')

C:\Users\Firat\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  import sys


FloorNumber filled.
RenovationYear filled.
Politics filled.
AreaProperty filled.
Done


## The Prediction:

In [6]:
# Address URL to request data from
url = 'http://127.0.0.1:5000/'

# Dicitonary of Elements send to API
file = df.to_dict('list')

# Make Post Request
response = requests.post(url=url, json=file)
predictions = response.json()
print('predictions:',predictions)

predictions: [1048502.0625]
